In [ ]:
# !wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
!wget -c "https://figshare.com/ndownloader/files/10798046"

In [ ]:
!gzip -d GoogleNews-vectors-negative300.bin.gz

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
!unzip -q glove.6B.zip

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import nltk
import gensim
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

# messages = pd.read_csv("con_test.csv", encoding='latin-1')
messages = pd.read_csv("super23.csv", encoding='latin-1')
# messages = pd.read_excel("punny.xlsx")

# messages = messages.drop(labels = ["Unnamed: 2", "Unnamed: 3"], axis = 1)
messages.columns = ["label", "message"]
messages['label'] = messages['label'].map({1:1,0:-1})
print (messages.tail())

In [ ]:
url = "GoogleNews-vectors-negative300.bin"
embeddings = gensim.models.KeyedVectors.load_word2vec_format(url, binary=True)

In [ ]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)

  return word_to_vec_map
embeddings = read_glove_vector('glove.6B.100d.txt') #glove.6B.50d.txt

In [ ]:
print (embeddings["win"])

In [ ]:
docs_vectors = pd.DataFrame() # creating empty final dataframe
stopwords = nltk.corpus.stopwords.words('english') # removing stop words
# for doc in messages['message'].str.lower().str.replace('[^a-z ]', ''): # looping through each document and cleaning it
for doc in messages['message'].str.lower():
    temp = pd.DataFrame()  # creating a temporary dataframe(store value for 1st doc & for 2nd doc remove the details of 1st & proced through 2nd and so on..)
    for word in doc.split(' '): # looping through each word of a single document and spliting through space
        if word not in stopwords: # if word is not present in stopwords then (try)
            try:
                word_vec = embeddings[word] # if word is present in embeddings(goole provides weights associate with words(300)) then proceed
                temp = temp.append(pd.Series(word_vec), ignore_index = True) # if word is present then append it to temporary dataframe
            except:
                pass
                # print("error")
    doc_vector = temp.mean() # take the average of each column(w0, w1, w2,........w300)
    docs_vectors = docs_vectors.append(doc_vector, ignore_index = True) # append each document value to the final dataframe
print (docs_vectors.shape)

docs_vectors['label'] = messages['label']

(65736, 100)


In [ ]:
docs_vectors = docs_vectors.dropna()
# docs_vectors.to_csv('WV_OC.csv', index=False)

In [ ]:
docs_vectors_train = docs_vectors.loc[0:52588, :]
# docs_vectors_train = docs_vectors.loc[0:5400, :]

In [ ]:
docs_vectors_train  = docs_vectors.dropna()

In [ ]:
# docs_vectors_train = pd.read_csv("WV_OC.csv", encoding='latin-1')
# print (docs_vectors_train.tail())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import OneClassSVM

train_x, test_x, train_y, test_y = train_test_split(docs_vectors_train.drop('label', axis = 1),
                                                   docs_vectors_train['label'],
                                                   test_size = 1,
                                                   random_state = 1)
print (train_x.shape, train_y.shape, test_x.shape, test_y.shape)

# model = AdaBoostClassifier(n_estimators=800, random_state = 1)

model = OneClassSVM(nu=0.2, kernel="rbf", gamma=0.1)
model.fit(train_x, train_y)

(64485, 100) (64485,) (1, 100) (1,)


OneClassSVM(gamma=0.1, nu=0.2)

In [ ]:
# docs_vectors_test = pd.read_csv("test101.csv", encoding='latin-1')
# print (docs_vectors_test.tail())

In [ ]:
docs_vectors_test = docs_vectors.loc[52589:65736, :] # 55461:55685 55686:55910 55911:56135 56136:56360 56361:56585 56586:56810
# docs_vectors_test = docs_vectors_test.dropna()
labels_test = docs_vectors_test.iloc[: , -1]
docs_vectors_test = docs_vectors_test.iloc[:,:-1]
# print ("test vects", docs_vectors_test)
# print ("test labels", labels_test)
# labels_test.to_csv('test2.csv', index=False)

In [ ]:
docs_vectors_test = docs_vectors_test.dropna()

In [ ]:
test_pred = model.predict(docs_vectors_test)
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
print(confusion_matrix(labels_test, test_pred))
# print(metrics.classification_report(labels_test, test_pred, digits=4))
print('Precision: ', precision_score(labels_test, test_pred))
print('Recall: ', recall_score(labels_test, test_pred))
print('Accuracy: ', accuracy_score(labels_test, test_pred))